# Laboratorio 5

1. Utilice los archivos pre-procesados que obtuvo en la práctica 3, mismos que almacenan la colección CACM y los queries.
2. Si en la práctica tres generó el vocabulario sin ordenarlo alfabéticamente ordénelo, tanto el normal como el reducido.
3. Represente cada documento empleando el modelo vectorial y el esquema de pesado tf. Recuerde que la dimensión de los vectores será igual al tamaño del vocabulario que obtuvo en la práctica 4. Trabajen con el vocabulario sin reducir y después reducido. Guarda las matrices generada, que representa los documentos de la colección, para utilizarla en prácticas posteriores.
4. Utiliza el archivo de queries y represéntelos también empleando el modelo vectorial y esquema de pesado tf. Con vocabulario normal y reducido. Guarda las matrices generadas.

Primero extraemos las palabras de documentos_final.txt


In [2]:
filename = "documentos_trunc.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
text = file.read()
file.close()
import re
words = re.split(r"\n", text)
doc_words = ""
doc_count = 0
for w in words:
    split = re.split(r"\|", w)
    if len(split) == 1:
        continue
    elif len(split) == 4:
        #doc_data[i]['d_num'] = split[0]
        doc_words += split[2] + " "
    doc_words += split[1] + " "
    doc_count += 1
doc_words = [x for x in (doc_words).split(" ") if x]
print(doc_words[:10])
print(doc_count)

['preliminari', 'report', 'intern', 'algebra', 'languag', 'extract', 'root', 'repeat', 'subtract', 'digit']
3204


Luego, importamos el vocabulario creado en la práctica 4.

In [3]:
filename = "vocabulario_trunc.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
vocabulary = file.read().split(", ")
file.close()
vocabulary = sorted(vocabulary)

Ahora, comenzamos los cálculos para la representación del vocabulario por medio del modelo vectorial.

In [4]:
# Primero, obtenemos las frecuencias de las palabras

from collections import Counter

dict_freq = dict(sorted(Counter(doc_words).items()))
dict_freq

#print(c.items())

{'1604a': 1,
 '16th': 1,
 '1970': 1,
 '2': 1,
 '263a': 1,
 '2f': 1,
 '2k': 2,
 '2m': 1,
 '2n': 13,
 '32k': 2,
 '3n': 1,
 '40k': 1,
 '44x': 1,
 '473l': 2,
 '4k': 1,
 '4n': 3,
 '4x16': 1,
 '5n': 1,
 '64k': 1,
 '7th': 1,
 '8k': 1,
 '8n': 1,
 '9n': 1,
 'a': 5,
 'a1': 28,
 'a2': 1,
 'a418': 1,
 'a419': 1,
 'a420': 1,
 'a421': 1,
 'a422': 1,
 'a423': 1,
 'a424': 1,
 'a425': 1,
 'a426': 1,
 'a427': 1,
 'a428': 1,
 'a429': 1,
 'a430': 1,
 'a431': 1,
 'a432': 1,
 'a433': 1,
 'a434': 1,
 'a435': 1,
 'a436': 1,
 'a437': 1,
 'a438': 1,
 'a439': 1,
 'a440': 1,
 'a441': 1,
 'a442': 1,
 'a443': 1,
 'a444': 1,
 'a445': 1,
 'a446': 1,
 'a447': 1,
 'a448': 1,
 'a449': 1,
 'a450': 1,
 'a451': 1,
 'a452': 1,
 'a453': 1,
 'a454': 1,
 'a455': 1,
 'a456': 1,
 'a457': 1,
 'a458': 1,
 'a459': 1,
 'a460': 1,
 'a461': 1,
 'a462': 1,
 'a463': 1,
 'a464': 1,
 'a465': 1,
 'a466': 1,
 'a467': 1,
 'a468': 1,
 'a469': 1,
 'a470': 1,
 'a471': 1,
 'a472': 1,
 'a473': 1,
 'a474': 1,
 'a475': 1,
 'a476': 1,
 'a477': 1,
 '

In [5]:
# separando las palabras por documento:
# text
from itertools import count
from operator import countOf


words = re.split(r"\n", text)
doc_words_per_doc = dict()
i = 1
for w in words:
    split = re.split(r"\|", w)
    index = str(i)
    if len(split) == 1:
        continue
    elif len(split) == 4:
        doc_words_per_doc[i] = [x for x in split[2].split(" ") if x]
    doc_words_per_doc[i] = [x for x in split[1].split(" ") if x]
    i += 1
doc_words_per_doc[1]

# contamos las frecuencias de cada término por documento
freq_per_doc = dict()
for d in doc_words_per_doc.keys():
    freq_per_doc[d] = dict().fromkeys(dict_freq.keys())
    for t in dict_freq.keys():
        freq_per_doc[d][t] = countOf(doc_words_per_doc[d], t)
        #print(t)
        
   
#{1: {"premiliniari": 1, "report": 3, ... , "algorithm": 0, ...}}



In [6]:
# dict_freq         -> vector q
# freq_per_doc[1]   -> vector d

Ahora lo mismo pero con el vocabulario reducido

In [7]:
import copy
filename = "vocabulario_reducido_trunc.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
red_voc = file.read().split(", ")
file.close()
red_voc = sorted(red_voc)

new_voc = copy.copy(dict_freq)
#freq_per_doc_red = { key : value for (key, value) in dict_freq if key in red_voc }
del_keys = [ key for key in new_voc if key not in red_voc ]
for key in del_keys:
     del new_voc[key]
print(len(dict_freq))
print(len(new_voc))


5178
4670


In [8]:
freq_per_doc_red = dict()
for d in doc_words_per_doc.keys():
    freq_per_doc_red[d] = dict().fromkeys(new_voc.keys())
    for t in new_voc.keys():
        freq_per_doc_red[d][t] = countOf(doc_words_per_doc[d], t)
        
# new_voc           -> vector q reducido
# freq_per_doc_red  -> matriz d reducido

## Con queries
### Vocabulario normal

In [9]:
filename = "querys_trunc.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
text = file.read()
file.close()
import re
words = re.split(r"\n", text)
quer_words_per_quer = dict()
i = 1
for w in words:
    split = re.split(r"\|", w)
    index = str(i)
    if len(split) == 1:
        continue
    elif len(split) == 4:
        quer_words_per_quer[i] = [x for x in split[2].split(" ") if x]
    quer_words_per_quer[i] = [x for x in split[1].split(" ") if x]
    i += 1
quer_words_per_quer[1]
print(len(quer_words_per_quer))

64


In [10]:
freq_per_quer = dict()
for d in quer_words_per_quer.keys():
    freq_per_quer[d] = dict().fromkeys(dict_freq.keys())
    for t in dict_freq.keys():
        freq_per_quer[d][t] = countOf(quer_words_per_quer[d], t)
        #print(t)

# dict_freq         -> vector q
# freq_per_quer     -> vector d

### Vocabulario reducido

In [11]:
freq_per_quer_red = dict()
for d in quer_words_per_quer.keys():
    freq_per_quer_red[d] = dict().fromkeys(new_voc.keys())
    for t in new_voc.keys():
        freq_per_quer_red[d][t] = countOf(quer_words_per_quer[d], t)


# new_voc         -> vector q
# freq_per_quer_red     -> vector d

In [13]:
print(freq_per_quer[1]["tss"])
#print(freq_per_quer_red[1]["tss"])

1


## Con tf-idf

Calcularemos primero un vector con los valores de frecuencia inversa del doc (idf)

In [14]:
#primero, calculamos N_t (# de docs con el término)
N_t = dict().fromkeys(dict_freq.keys())
N_t = { x : 0 for x in N_t}
for i in N_t.keys():
    for j in freq_per_doc.keys():
        if freq_per_doc[j][i] != 0:
            N_t[i] += 1

In [16]:
contract = 0
for i in freq_per_doc:
    if freq_per_doc[i]["contract"] != 0:
        contract += 1
contract

1